In [ ]:
# Imports

import numpy as np
import scanpy as sc
import pandas as pd
import os
import bbknn
from pathlib import Path
# %%time

In [ ]:
# Single Cell settings

sc.settings.verbosity = 4
sc.settings.set_figure_params(80)
sc.settings._file_format_figs = 'pdf'
sc.settings.savefigs = False
use_first_n_samples = 0
full_sparse = False

# Basic QC workflow

In [ ]:
adatas = []
data_names = ['control','lf','lps','pic']
main_path = '/Code/data/single_cell/animals/turtle/turtle_3/'
paths = ['\\control','\\lf','\\lps','\\pic']           #...etc - Folders with matrix, features and barcodes gz files.
for path in paths:
    adata = sc.read_10x_mtx(main_path + path, var_names='gene_symbols',        # use gene symbols for the variable names (variables-axis index)
        cache=False)  
    sc.logging.print_memory_usage()
    print(adata.shape)
    adatas.append(adata)

In [ ]:
# os.getcwd()

In [ ]:
adata = adatas[0].concatenate(adatas[1:], batch_categories = data_names, batch_key='treatment')

In [ ]:
genes = pd.read_csv('/Code/data/single_cell/animals/turtle/turtle_3/eggnog_combined.csv')
genes.dropna(subset = ['turtle_3_gene_id'], inplace=True)

genes_dict = dict(zip(genes['turtle_3_gene_id'].astype('str'), genes.eggnog_name.astype('str')))

dict_multi, dict_uni = {}, {}
for d in genes_dict:
    if len(d.split(',')) != 1:
        dict_multi[d] = genes_dict[d]
    else:
        dict_uni[d] =  genes_dict[d]

multi_result_dict = {}

for key_str, value in dict_multi.items():
    # Convert the string key to a set
    key_set = set(key_str.strip("{}").replace("'", "").split(", "))

    # Iterate through the elements in the set and create individual keys
    for element in key_set:
        multi_result_dict[element] = value

genes_dict = {**dict_uni, **multi_result_dict}

genes_dict

In [ ]:
# adata.var.set_index('gene_ids', inplace=True) 
# adata.var['gene_ids'] = adata.var.index

# adata.var.drop(columns='feature_types', inplace=True)
# adata.var.rename(genes_dict, inplace=True)

In [ ]:
# This code identifies duplicate variable names, iterates through them, and adds suffixes to make them unique from the first occurrence.
# Please make sure to replace adata with your actual AnnData object.

# Keep in mind that Scanpy's functionality may evolve, so it's a good idea to check the documentation or release notes
# for any updates related to making variable names unique starting from the first occurrence in more recent versions of Scanpy.

# Get a Series of variable (gene) names
var_names_series = pd.Series(adata.var_names)

# Identify duplicate variable names
duplicate_var_names = var_names_series[var_names_series.duplicated(keep='first')]

# Iterate through duplicates and add suffixes to make them unique from the first occurrence
for duplicate_name in duplicate_var_names:
    count = 1
    indices = var_names_series[var_names_series == duplicate_name].index
    for idx in indices:
        var_names_series[idx] = f'{duplicate_name}_{count}'
        count += 1

# Update the AnnData object with the unique variable names
adata.var_names = var_names_series.values

adata.var_names

## Creating a raw adata annotated with the metadata created in downstream processing 
From the future regarding that script 

In [ ]:
# adata = sc.read(r"C:\Users\TzachiHNB5\Desktop\reptiles\turtle\turtle_raw_with_metadata.h5ad")

In [ ]:
# adata.obs['inter_species_ann'] = adata.obs['ce']

In [ ]:
# meta = pd.read_csv('turtle_meta.csv',index_col=0)
# meta

In [ ]:
# adata = adata[meta.index]

In [ ]:
# adata

In [ ]:
# adata.obs = meta

In [ ]:
# adata.obs.drop(columns = ['leiden'],inplace=True)

In [ ]:
# adata.write('turtle_raw_with_metadata.h5ad')

In [ ]:
# pwd

### Genes and cells filtration 


Show those genes that yield the highest fraction of counts in each single cell, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Basic filtering

In [ ]:
adata_copy = adata.copy()

In [ ]:
sc.pp.filter_cells(adata, min_genes=100) 
sc.pp.filter_genes(adata, min_cells=3)
print(adata.shape) 

### Mitochondrial QC and general measures

Check if genes are annotated as mt by running:
- GENES= list(adata.var.index[adata.var.index.str.startswith('mt-'.upper())])
- GENES

In case the genes are not annotated as 'MT-'' (Like in bats), run:

- dict_replace = {'COX1':'MT-COX1','COX2':'MT-COX2'...etc}
- adata.var.rename(dict_replace, inplace = True)

In [ ]:
dict_replace = {'COX1':'MT-COX1','COX2':'MT-COX2','COX3':'MT-COX3','ND1':'MT-ND1-1','ND1':'MT-ND1-2','ND2':'MT-ND2',
                'ND3':'MT-ND3','ND4':'MT-ND4','ND5':'MT-ND5','ND6':'MT-ND6','ND4L-1':'MT-ND4L','ATP6':'MT-ATP6','ATP8':'MT-ATP8',
                'CYTB':'MT-CYTB'}
adata.var.rename(dict_replace, inplace = True)
# ONLY MT-ATP6, MT-ND1, MT-ND4L

In [ ]:
adata.var['MT'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)

A violin plot of some of the computed quality measures:

- the number of genes expressed in the count matrix
- the total counts per cell
- the percentage of counts in mitochondrial genes

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='treatment')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_MT', color='treatment')
sc.pl.scatter(adata, x='total_counts', y='n_genes', color='treatment')

Actually do the filtering by slicing the AnnData object - By pct_counts_MT  and by total_counts /n_genes_by_counts or even n_genes

In [ ]:
adata = adata[adata.obs.pct_counts_MT < 20] 
adata = adata[adata.obs.total_counts < 5000, :] # If filtering outliers (<0.1% of cells)

### Doublet analysis and filtering

In [ ]:
import scrublet as scr

def scrub(adatas,adata,adata_names): # based on raw individual samples. 
    print('Before scrublet: ', adata.shape[0])
    doub_index =[]
    barcodes =[]
    for data,name in zip(adatas,adata_names):
        data.raw = data
        sc.pp.normalize_total(data, target_sum=1e4)
        sc.pp.log1p(data)
        scrub = scr.Scrublet(data.raw.X)
        data.obs['doublet_scores'], data.obs['predicted_doublets'] = scrub.scrub_doublets()
        scrub.plot_histogram()
        print('Doublets'+name +' :',data.obs[data.obs['doublet_scores'] >0.25].shape[0])
        barcodes = data.obs[data.obs['doublet_scores'] <0.25].index.to_list()
        for barcode in barcodes:
            doub_index.append(barcode+'-'+name)
    
    adata = adata[adata.obs.index.isin(doub_index)]
    print('After scrublet: ',adata.shape[0])
    return adata

In [ ]:
adata = scrub(adatas,adata,data_names)

### Cell cycle scoring

download Cell cycle txt: https://github.com/scverse/scanpy_usage/blob/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt

In [ ]:
cell_cycle_genes = [x.strip() for x in open(r'/Code/data/single_cell/animals/turtle/turtle_3/regev_lab_cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]

In [ ]:
cell_cycle_adata = adata.copy()

sc.pp.normalize_per_cell(cell_cycle_adata, counts_per_cell_after=1e4)
sc.pp.log1p(cell_cycle_adata)
sc.pp.scale(cell_cycle_adata)
sc.tl.score_genes_cell_cycle(cell_cycle_adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata_cc_genes = cell_cycle_adata[:, cell_cycle_genes].copy()
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase')
adata.obs['S_score'] = cell_cycle_adata.obs['S_score'].copy()
adata.obs['G2M_score'] = cell_cycle_adata.obs['G2M_score'].copy()
adata.obs['phase'] = cell_cycle_adata.obs['phase'].copy()

### Saving adata

In [ ]:
adata.write(r'/Code/data/single_cell/animals/turtle/turtle_3/''turtle_after_qc.h5ad')